In [1]:
import pandas as pd

Este notebook es para unir los resultados de los 3 mecanismos alternativos de selección:

- CNO 3 digitos
- IPO no ponderado
- Transformación convexa 

In [2]:
def group_data(data, keys = ["Ruta habilitada", "Ocupacion"]):
    resumen = data.groupby(
    by=keys
    ).agg(
        IPO=("IPO", "mean"),
        cupos_habilitados=("cupos_habilitados", "sum"),
        monto_asignado = ("monto_asignado","sum")
    ).reset_index()
    return resumen

In [3]:
# Función para renombrar columnas con sufijo, excepto las claves
def renombrar_columnas(df, sufijo, claves):
    return df.rename(columns={
        col: f"{col}_{sufijo}" for col in df.columns if col not in claves
    })


In [4]:
pd.set_option('display.float_format', '{:,.2f}'.format)

In [5]:
# Cargar archivos
cno3 = pd.read_excel("Export/CNO3/resumen_via_cno3.xlsx")
ipo_no_ponderado = pd.read_excel("Export/SinPonderar/resumen_via_sinponderar.xlsx")
convexa = pd.read_excel("Export/Convexa/resumen_via_convexa.xlsx")
original = pd.read_excel("Export/resumen_via_original.xlsx")

cno3 = cno3.drop(columns=["Unnamed: 0"], errors='ignore')
ipo_no_ponderado = ipo_no_ponderado.drop(columns=["Unnamed: 0"], errors='ignore')
convexa = convexa.drop(columns=["Unnamed: 0"], errors='ignore')
original = original.drop(columns=["Unnamed: 0"], errors='ignore')

In [7]:
cno3_grouped = group_data(cno3)
ipo_no_ponderado_grouped = group_data(ipo_no_ponderado)
convexa_grouped = group_data(convexa)
original_grouped = group_data(original)

In [9]:
# Llaves de unión (sin 'nombre_institucion')
keys = ["Ruta habilitada", "Ocupacion"]

# Renombrar columnas
original_grouped = renombrar_columnas(original_grouped, "original", keys)
cno3_grouped = renombrar_columnas(cno3_grouped, "cno3", keys)
ipo_no_ponderado_grouped = renombrar_columnas(ipo_no_ponderado_grouped, "ipo", keys)
convexa_grouped = renombrar_columnas(convexa_grouped, "convexa", keys)

# Hacer merge progresivamente
df_merge = original_grouped.merge(cno3_grouped, on=keys, how='outer')
df_merge = df_merge.merge(ipo_no_ponderado_grouped, on=keys, how='outer')
df_final = df_merge.merge(convexa_grouped, on=keys, how='outer')

# Ordenar
df_final = df_final.sort_values(
    by=["Ruta habilitada","monto_asignado_original", "monto_asignado_cno3", "monto_asignado_ipo", "monto_asignado_convexa"],
    ascending=[True,False, False, False, False]
)

# Quitar decimales
cols = ["monto_asignado_original","monto_asignado_convexa", "monto_asignado_ipo", "monto_asignado_cno3"]
df_final[cols] = df_final[cols].fillna(0).astype(int)

df_final = df_final.reset_index(drop=True)

In [10]:
df_final.to_excel("asignaciones_por_ocupacion_mecanismos_alternativos.xlsx")

In [11]:
df_final.head()

,Ruta habilitada,Ocupacion,IPO_original,cupos_habilitados_original,monto_asignado_original,IPO_cno3,cupos_habilitados_cno3,monto_asignado_cno3,IPO_ipo,cupos_habilitados_ipo,monto_asignado_ipo,IPO_convexa,cupos_habilitados_convexa,monto_asignado_convexa
0,Antiguos,"Auxiliares técnicos de instalación, mantenimie...",0.50,42,227058468,0.50,42,227058468,0.50,45,243276930,0.50,39,210840006
1,Antiguos,Cocineros,0.55,61,215523078,0.55,61,215523078,0.55,61,215523078,0.55,62,219056243
2,Antiguos,Mercaderistas e impulsadores,0.56,36,173176347,0.56,36,173176347,0.56,22,105829990,0.56,37,177986801
3,Antiguos,Auxiliares de educación para la primera infancia,0.48,14,153661017,0.48,14,153661017,0.48,12,131709443,0.48,12,131709443
4,Antiguos,Auxiliares administrativos en salud,0.62,17,110572617,0.62,17,110572617,0.62,18,117076889,0.62,19,123581161
